In [ ]:
%%capture
!pip install PyPDF2==3.0.0
!pip install cohere
!pip install langchain langchain-cohere langchain-openai
!pip install pytesseract
!pip install tkinter
!pip install pdf2image langsmith

In [ ]:
from langchain_cohere.llms import Cohere
import os
import getpass

os.environ["COHERE_API_KEY"] = "2zPsEcvQ1tzELqmw0ORVcVEX51H5p7KXLrpaqmrz"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"

In [ ]:
import tkinter as tk
from tkinter import filedialog


def select_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename()

    return file_path

selected_file_path = select_file()


In [ ]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

In [ ]:
from pdf2image import convert_from_path
import pytesseract
from PIL import Image

def apply_ocr_to_pdf(pdf_path):
    images = convert_from_path(pdf_path)
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image)
    return text

In [ ]:
# Function to determine whether to extract text or apply OCR
def process_pdf(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    if len(text) < 100:
        text = apply_ocr_to_pdf(pdf_path)
    return text

In [ ]:
from langchain_core.prompts import PromptTemplate


prompt_template = PromptTemplate.from_template(
    """
    Given the following context:
    
    {context}
    
    Please answer the following question based on the above context:
    
    {question}
    
    Your answer should be concise and focus on the most important details. Make sure the answer is in JSON format. If you don't know a specific value, keep it empty. The dates should be specified like this: 25th January 2011.
    """
)

In [ ]:
from langchain_cohere import ChatCohere

model = ChatCohere(model="command-r-plus")

In [ ]:
from langsmith import Client

# QA
inputs = [
    "What are the names of the passengers?",
    "What is the airline's name and flight number?",
    "Can you provide me the flight routes along with the time?",
    "Can you provide me with the ticket number of the passengers?",
    "What are the flight duration for all the flights mentioned?",
    "What are the baggage limits?",
    "What are the date of birth of the passengers?",
    "What are the ticket numbers?"
]

outputs = [
    "",
    "",
    "",
    "",
    "",
    "",
    "",
    "",
]

qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]

# Create dataset
client = Client()
dataset_name = "ticket_test_evaluation_hallucination"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="QA pairs about ticket.",
)
client.create_examples(
    inputs=[{"question": q} for q in inputs],
    outputs=[{"answer": a} for a in outputs],
    dataset_id=dataset.id,
)

In [ ]:
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = prompt_template | model | parser

In [ ]:
def answer_with_context(example: dict):
    context = process_pdf(selected_file_path)
    result = chain.invoke({"context": context, "question": example["question"]})
    return {"answer": result, "contexts": context}

In [ ]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

answer_hallucination_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "criteria": {
            "accuracy": """Is the Assistant's Answer grounded in the Ground Truth documentation? A score of [[1]] means that the
            Assistant answer contains is not at all based upon / grounded in the Groun Truth documentation. A score of [[5]] means 
            that the Assistant answer contains some information (e.g., a hallucination) that is not captured in the Ground Truth 
            documentation. A score of [[10]] means that the Assistant answer is fully based upon the in the Ground Truth documentation."""
        },
        # If you want the score to be saved on a scale from 0 to 1
        "normalize_by": 10,
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["contexts"],
        "input": example.inputs["question"],
    },
)

In [ ]:
dataset_name = "ticket_test_evaluation_hallucination"
experiment_results = evaluate(
    answer_with_context,
    data=dataset_name,
    evaluators=[answer_hallucination_evaluator],
    experiment_prefix="rag-qa-cohere-hallucination",
    # Any experiment metadata can be specified here
    metadata={
        "variant": "ticket_test_evaluation_hallucination_cohere",
    },
)